In [1]:
import os
os.curdir
os.chdir('/Users/manojravikumar/Public/60162417/05-tutorials/14-aiml/machine-learning-practice/supervised')

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

In [12]:
df = pd.read_csv('Travel.csv')
df1 = df.copy()

In [13]:
df1.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [14]:
df1.isna().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [15]:
print(df["Gender"].value_counts().to_dict())
df1["Gender"]=df1["Gender"].replace("Fe Male","Female")
print(df1["Gender"].value_counts().to_dict())
print(df["MaritalStatus"].value_counts().to_dict())
df1["MaritalStatus"]=df1["MaritalStatus"].replace("Unmarried","Single")
print(df1["MaritalStatus"].value_counts().to_dict())
features_with_na=[features for features in df1.columns if df1[features].isnull().sum()>=1]
for feature in features_with_na:
    print(feature,np.round(df1[feature].isnull().mean()*100,5), '% missing values')
df1[features_with_na].select_dtypes(exclude='object').describe()

#PreferredPropertyStar
df1.PreferredPropertyStar.fillna(df1.PreferredPropertyStar.mode()[0], inplace=True)

#TypeofContract
df1.TypeofContact.fillna(df1.TypeofContact.mode()[0], inplace=True)

#NumberOfChildrenVisiting
df1.NumberOfChildrenVisiting.fillna(df1.NumberOfChildrenVisiting.mode()[0], inplace=True)

#Age
df1.Age.fillna(df1.Age.median(), inplace=True)

#DurationOfPitch
df1.DurationOfPitch.fillna(df1.DurationOfPitch.median(), inplace=True)

#NumberOfFollowups
df1.NumberOfFollowups.fillna(df1.NumberOfFollowups.mode()[0], inplace=True)

#NumberOfTrips
df1.NumberOfTrips.fillna(df1.NumberOfTrips.median(), inplace=True)

#MonthlyIncome
df1.MonthlyIncome.fillna(df1.MonthlyIncome.median(), inplace=True)



df1.drop('CustomerID', inplace=True, axis=1)
df1['TotalVisiting'] = df1['NumberOfPersonVisiting'] + df1['NumberOfChildrenVisiting']
df1.drop(columns=['NumberOfPersonVisiting', 'NumberOfChildrenVisiting'], axis=1, inplace=True)


num_features = [feature for feature in df1.columns if df1[feature].dtype != 'O']
print('Num of Numerical Features :', len(num_features), "; Columns : ", ",".join(num_features))
cat_features = [feature for feature in df1.columns if df1[feature].dtype == 'O']
print('Num of Categorical Features :', len(cat_features), "; Columns : ", ",".join(cat_features))
discrete_features=[feature for feature in num_features if len(df1[feature].unique())<=25]
print('Num of Discrete Features :',len(discrete_features), "; Columns : ", ",".join(discrete_features))
continuous_features=[feature for feature in num_features if feature not in discrete_features]
print('Num of Continuous Features :',len(continuous_features), "; Columns : ", ",".join(continuous_features))


{'Male': 2916, 'Female': 1817, 'Fe Male': 155}
{'Male': 2916, 'Female': 1972}
{'Married': 2340, 'Divorced': 950, 'Single': 916, 'Unmarried': 682}
{'Married': 2340, 'Single': 1598, 'Divorced': 950}
Age 4.62357 % missing values
TypeofContact 0.51146 % missing values
DurationOfPitch 5.13502 % missing values
NumberOfFollowups 0.92062 % missing values
PreferredPropertyStar 0.53191 % missing values
NumberOfTrips 2.86416 % missing values
NumberOfChildrenVisiting 1.35025 % missing values
MonthlyIncome 4.76678 % missing values
Num of Numerical Features : 12 ; Columns :  ProdTaken,Age,CityTier,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,MonthlyIncome,TotalVisiting
Num of Categorical Features : 6 ; Columns :  TypeofContact,Occupation,Gender,ProductPitched,MaritalStatus,Designation
Num of Discrete Features : 9 ; Columns :  ProdTaken,CityTier,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,

In [19]:

y1 = df1['ProdTaken']
X1 = df1.drop(['ProdTaken'], axis=1)


from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(X1,y1,test_size=0.2,random_state=42)
print("X1_train shape is ",X1_train.shape, " X1_test shape is ",X1_test.shape)
cat_features = X1.select_dtypes(include="object").columns
print(" Category Features are :: ",",".join(cat_features))
num_features = X1.select_dtypes(exclude="object").columns
print(" Number Features are :: ",",".join(num_features))
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')
preprocessor = ColumnTransformer(
    [
         ("OneHotEncoder", oh_transformer, cat_features),
          ("StandardScaler", numeric_transformer, num_features)
    ]
)

X1_train=preprocessor.fit_transform(X1_train)
X1_test=preprocessor.transform(X1_test)

X1_train shape is  (3910, 17)  X1_test shape is  (978, 17)
 Category Features are ::  TypeofContact,Occupation,Gender,ProductPitched,MaritalStatus,Designation
 Number Features are ::  Age,CityTier,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,MonthlyIncome,TotalVisiting


In [20]:
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, precision_score, recall_score, f1_score, roc_auc_score,roc_curve 

models={
    "Adaboost":AdaBoostClassifier(),
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X1_train, y1_train) # Train model

    # Make predictions
    y1_train_pred = model.predict(X1_train)
    y1_test_pred = model.predict(X1_test)

    # Training set performance
    model_train_accuracy = accuracy_score(y1_train, y1_train_pred) # Calculate Accuracy
    model_train_f1 = f1_score(y1_train, y1_train_pred, average='weighted') # Calculate F1-score
    model_train_precision = precision_score(y1_train, y1_train_pred) # Calculate Precision
    model_train_recall = recall_score(y1_train, y1_train_pred) # Calculate Recall
    model_train_rocauc_score = roc_auc_score(y1_train, y1_train_pred)


    # Test set performance
    model_test_accuracy = accuracy_score(y1_test, y1_test_pred) # Calculate Accuracy
    model_test_f1 = f1_score(y1_test, y1_test_pred, average='weighted') # Calculate F1-score
    model_test_precision = precision_score(y1_test, y1_test_pred) # Calculate Precision
    model_test_recall = recall_score(y1_test, y1_test_pred) # Calculate Recall
    model_test_rocauc_score = roc_auc_score(y1_test, y1_test_pred) #Calculate Roc


    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print('- F1 score: {:.4f}'.format(model_train_f1))
    
    print('- Precision: {:.4f}'.format(model_train_precision))
    print('- Recall: {:.4f}'.format(model_train_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))

    
    
    print('----------------------------------')
    
    print('Model performance for Test set')
    print('- Accuracy: {:.4f}'.format(model_test_accuracy))
    print('- F1 score: {:.4f}'.format(model_test_f1))
    print('- Precision: {:.4f}'.format(model_test_precision))
    print('- Recall: {:.4f}'.format(model_test_recall))
    print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))

    
    print('='*35)
    print('\n')

Adaboost
Model performance for Training set
- Accuracy: 0.8478
- F1 score: 0.8146
- Precision: 0.7815
- Recall: 0.2551
- Roc Auc Score: 0.6194
----------------------------------
Model performance for Test set
- Accuracy: 0.8354
- F1 score: 0.7987
- Precision: 0.7500
- Recall: 0.2356
- Roc Auc Score: 0.6083


